In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
# dp
from sklearn.model_selection import train_test_split
import os

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 6, 300, 1]) # dp

    # Convolutional Layer #1 6*300*32
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1 3*150*32
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2 3*150*64 
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    # 3x2 => 1*75*64
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[3, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 1 * 75 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=14) # 14 классов у Самодурова

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    # Load training and eval data
# dp
    cwd = r"C:\astrodata"
    os.chdir(cwd)
    compressedNpDataFile = r'14savedNpAstro0.npz'
    loaded = np.load(compressedNpDataFile)
    X_train, X_test, y_train, y_test = train_test_split(loaded['D'], loaded['L'], test_size=0.33, random_state=42)
    loaded.close()
# dp    
    #mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = X_train   #mnist.train.images # Returns np.array
    train_labels = y_train #np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = X_test     #mnist.test.images # Returns np.array
    eval_labels = y_test   #np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator
    astro_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=r"C:\astrodata\CNN\model\astro_convnet_model")
    
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    astro_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = astro_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\astrodata\\CNN\\model\\astro_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000287A16ACE48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updatin

INFO:tensorflow:loss = 2.047394037246704, step = 6816 (27.732 sec)
INFO:tensorflow:probabilities = [[0.25720289 0.08332962 0.04995184 ... 0.04010288 0.04023262 0.03989601]
 [0.25720289 0.08332962 0.04995184 ... 0.04010288 0.04023262 0.03989601]
 [0.25720289 0.08332962 0.04995184 ... 0.04010288 0.04023262 0.03989601]
 ...
 [0.25720289 0.08332962 0.04995184 ... 0.04010288 0.04023262 0.03989601]
 [0.25720289 0.08332962 0.04995184 ... 0.04010288 0.04023262 0.03989601]
 [0.25720289 0.08332962 0.04995184 ... 0.04010288 0.04023262 0.03989601]] (13.836 sec)
INFO:tensorflow:global_step/sec: 3.61332
INFO:tensorflow:probabilities = [[0.25817123 0.08335229 0.04982041 ... 0.03993994 0.04007009 0.03973273]
 [0.25817123 0.08335229 0.04982041 ... 0.03993994 0.04007009 0.03973273]
 [0.25817123 0.08335229 0.04982041 ... 0.03993994 0.04007009 0.03973273]
 ...
 [0.25817123 0.08335229 0.04982041 ... 0.03993994 0.04007009 0.03973273]
 [0.25817123 0.08335229 0.04982041 ... 0.03993994 0.04007009 0.03973273]
 

INFO:tensorflow:probabilities = [[0.27219893 0.08376181 0.0480338  ... 0.03775978 0.03789597 0.0375465 ]
 [0.27219893 0.08376181 0.0480338  ... 0.03775978 0.03789597 0.0375465 ]
 [0.27219893 0.08376181 0.0480338  ... 0.03775978 0.03789597 0.0375465 ]
 ...
 [0.27219893 0.08376181 0.0480338  ... 0.03775978 0.03789597 0.0375465 ]
 [0.27219893 0.08376181 0.0480338  ... 0.03775978 0.03789597 0.0375465 ]
 [0.27219893 0.08376181 0.0480338  ... 0.03775978 0.03789597 0.0375465 ]] (13.718 sec)
INFO:tensorflow:loss = 1.883639931678772, step = 7616 (27.447 sec)
INFO:tensorflow:probabilities = [[0.27319927 0.08377156 0.04791556 ... 0.03761017 0.03774745 0.03739739]
 [0.27319927 0.08377156 0.04791556 ... 0.03761017 0.03774745 0.03739739]
 [0.27319927 0.08377156 0.04791556 ... 0.03761017 0.03774745 0.03739739]
 ...
 [0.27319927 0.08377156 0.04791556 ... 0.03761017 0.03774745 0.03739739]
 [0.27319927 0.08377156 0.04791556 ... 0.03761017 0.03774745 0.03739739]
 [0.27319927 0.08377156 0.04791556 ... 0.0

INFO:tensorflow:global_step/sec: 3.56895
INFO:tensorflow:probabilities = [[0.2862202  0.08419102 0.04619896 ... 0.03551254 0.03565025 0.03529369]
 [0.2862202  0.08419102 0.04619896 ... 0.03551254 0.03565025 0.03529369]
 [0.2862202  0.08419102 0.04619896 ... 0.03551254 0.03565025 0.03529369]
 ...
 [0.2862202  0.08419102 0.04619896 ... 0.03551254 0.03565025 0.03529369]
 [0.2862202  0.08419102 0.04619896 ... 0.03551254 0.03565025 0.03529369]
 [0.2862202  0.08419102 0.04619896 ... 0.03551254 0.03565025 0.03529369]] (13.954 sec)
INFO:tensorflow:loss = 1.928734540939331, step = 8416 (27.989 sec)
INFO:tensorflow:Saving checkpoints for 8464 into C:\astrodata\CNN\model\astro_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[0.28707384 0.0842237  0.04608    ... 0.03537879 0.03551715 0.03515938]
 [0.28707384 0.0842237  0.04608    ... 0.03537879 0.03551715 0.03515938]
 [0.28707384 0.0842237  0.04608    ... 0.03537879 0.03551715 0.03515938]
 ...
 [0.28707384 0.0842237  0.04608    ... 0.03

INFO:tensorflow:loss = 1.8327144384384155, step = 9116 (27.928 sec)
INFO:tensorflow:probabilities = [[0.29734167 0.08477621 0.04464137 ... 0.03362474 0.03376642 0.03340133]
 [0.29734167 0.08477621 0.04464137 ... 0.03362474 0.03376642 0.03340133]
 [0.29734167 0.08477621 0.04464137 ... 0.03362474 0.03376642 0.03340133]
 ...
 [0.29734167 0.08477621 0.04464137 ... 0.03362474 0.03376642 0.03340133]
 [0.29734167 0.08477621 0.04464137 ... 0.03362474 0.03376642 0.03340133]
 [0.29734167 0.08477621 0.04464137 ... 0.03362474 0.03376642 0.03340133]] (13.975 sec)
INFO:tensorflow:global_step/sec: 3.57725
INFO:tensorflow:probabilities = [[0.29803629 0.08481275 0.04454641 ... 0.03350564 0.03364759 0.03328207]
 [0.29803629 0.08481275 0.04454641 ... 0.03350564 0.03364759 0.03328207]
 [0.29803629 0.08481275 0.04454641 ... 0.03350564 0.03364759 0.03328207]
 ...
 [0.29803629 0.08481275 0.04454641 ... 0.03350564 0.03364759 0.03328207]
 [0.29803629 0.08481275 0.04454641 ... 0.03350564 0.03364759 0.03328207]


...

INFO:tensorflow:loss = 1.5122997760772705, step = 24416 (27.388 sec)
INFO:tensorflow:probabilities = [[0.36653167 0.10582517 0.03258421 ... 0.01680756 0.0169817  0.01654029]
 [0.36653167 0.10582517 0.03258421 ... 0.01680756 0.0169817  0.01654029]
 [0.36653167 0.10582517 0.03258421 ... 0.01680756 0.0169817  0.01654029]
 ...
 [0.36653167 0.10582517 0.03258421 ... 0.01680756 0.0169817  0.01654029]
 [0.36653167 0.10582517 0.03258421 ... 0.01680756 0.0169817  0.01654029]
 [0.36653167 0.10582517 0.03258421 ... 0.01680756 0.0169817  0.01654029]] (13.696 sec)
INFO:tensorflow:global_step/sec: 3.64959
INFO:tensorflow:probabilities = [[0.36645578 0.10589544 0.03257471 ... 0.01678351 0.01695742 0.01651618]
 [0.36645578 0.10589544 0.03257471 ... 0.01678351 0.01695742 0.01651618]
 [0.36645578 0.10589544 0.03257471 ... 0.01678351 0.01695742 0.01651618]
 ...
 [0.36645578 0.10589544 0.03257471 ... 0.01678351 0.01695742 0.01651618]
 [0.36645578 0.10589544 0.03257471 ... 0.01678351 0.01695742 0.01651618]

INFO:tensorflow:probabilities = [[0.36708239 0.10682403 0.03241068 ... 0.01644367 0.01661868 0.01617525]
 [0.36708239 0.10682403 0.03241068 ... 0.01644367 0.01661868 0.01617525]
 [0.36708239 0.10682403 0.03241068 ... 0.01644367 0.01661868 0.01617525]
 ...
 [0.36708239 0.10682403 0.03241068 ... 0.01644367 0.01661868 0.01617525]
 [0.36708239 0.10682403 0.03241068 ... 0.01644367 0.01661868 0.01617525]
 [0.36708239 0.10682403 0.03241068 ... 0.01644367 0.01661868 0.01617525]] (13.681 sec)
INFO:tensorflow:loss = 1.6645079851150513, step = 25216 (27.365 sec)
INFO:tensorflow:probabilities = [[0.3668853  0.1069617  0.03240395 ... 0.0164238  0.01659779 0.01615447]
 [0.3668853  0.1069617  0.03240395 ... 0.0164238  0.01659779 0.01615447]
 [0.3668853  0.1069617  0.03240395 ... 0.0164238  0.01659779 0.01615447]
 ...
 [0.3668853  0.1069617  0.03240395 ... 0.0164238  0.01659779 0.01615447]
 [0.3668853  0.1069617  0.03240395 ... 0.0164238  0.01659779 0.01615447]
 [0.3668853  0.1069617  0.03240395 ... 0

INFO:tensorflow:loss = 1.7826008796691895, step = 25916 (28.677 sec)
INFO:tensorflow:probabilities = [[0.36765114 0.10785703 0.03226689 ... 0.01610202 0.01627776 0.01583145]
 [0.36765114 0.10785703 0.03226689 ... 0.01610202 0.01627776 0.01583145]
 [0.36765114 0.10785703 0.03226689 ... 0.01610202 0.01627776 0.01583145]
 ...
 [0.36765114 0.10785703 0.03226689 ... 0.01610202 0.01627776 0.01583145]
 [0.36765114 0.10785703 0.03226689 ... 0.01610202 0.01627776 0.01583145]
 [0.36765114 0.10785703 0.03226689 ... 0.01610202 0.01627776 0.01583145]] (13.689 sec)
INFO:tensorflow:global_step/sec: 3.65617
INFO:tensorflow:probabilities = [[0.36785282 0.10791552 0.03225634 ... 0.01607807 0.01625455 0.01580764]
 [0.36785282 0.10791552 0.03225634 ... 0.01607807 0.01625455 0.01580764]
 [0.36785282 0.10791552 0.03225634 ... 0.01607807 0.01625455 0.01580764]
 ...
 [0.36785282 0.10791552 0.03225634 ... 0.01607807 0.01625455 0.01580764]
 [0.36785282 0.10791552 0.03225634 ... 0.01607807 0.01625455 0.01580764]

SystemExit: 

C:\Users\dvperv\Anaconda3\envs\tf36gpu\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
